<a href="https://colab.research.google.com/github/zknepp1/cs5293sp23-project0/blob/main/project0_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [81]:
import urllib
import PyPDF2
import urllib.request
import os
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse
import sqlite3
import io

In [34]:
def get_pdf(url):
  req = urllib.request.Request(url, headers={'User-Agent' : "Magic Browser"})
  remote_file = urllib.request.urlopen(req).read()
  remote_file_bytes = io.BytesIO(remote_file)
  pdfdoc = PyPDF2.PdfReader(remote_file_bytes)

  lines = []
  for page in pdfdoc.pages:
    # Extracting text from page
    # And splitting it into chunks of lines
    text = page.extract_text().split("\n")
    lines.append(text)

  lines_separated =[]
  count = 0
  for line in lines:
    for l in line:
      lines_separated.append(l)
      count = count + 1

  return lines_separated


In [35]:
url = "https://www.normanok.gov/sites/default/files/documents/2023-01/2023-01-01_daily_incident_summary.pdf"
p = get_pdf(url)


In [90]:
regex_datetime = r"(\d{1})\/(\d{1})\/(\d{4}) ([0-9]+):(\d{2})"
regex_incident_num = r"(\d{4})-(\d{8})"

#( \d{1,5}. ([A-Z]|[1-9])+ [A-Z]+ (([A-Z]+)\b)?)|( [A-Z]+ [A-Z]+ [A-Z]+ \/ [A-Z]+ [A-Z]+ [A-Z]+)|( [A-Z]+ [A-Z]+ \/ [A-Z]+ [A-Z]+)|([A-Z] [A-Z]+ [A-Z]+ \/ \d+[A-Z]+ [A-Z]+ [A-Z]+)|([1-9]+[A-Z]+ [A-Z]+ [A-Z]+ \/ [A-Z]+ \d)|(\d\d\d\d [A-Z]+ \d+\w+ \w+)|([A-Z] [A-Z]+ [A-Z]+ \d [A-Z]+ \/ [A-Z] [A-Z]+ [A-Z]+)|([A-Z]+ [A-Z]+ \/ \d+[A-Z]+ [A-Z]+ [A-Z]+)
#( \d{1,5}. ([A-Z]|[1-9])+ [A-Z]+ (([A-Z]+)\b)?)|( [A-Z]+ [A-Z]+ [A-Z]+ \/ [A-Z]+ [A-Z]+ [A-Z]+)|( [A-Z]+ [A-Z]+ \/ [A-Z]+ [A-Z]+)|([A-Z] [A-Z]+ [A-Z]+ \/ \d+[A-Z]+ [A-Z]+ [A-Z]+)|([1-9]+[A-Z]+ [A-Z]+ [A-Z]+ \/ [A-Z]+ \d)|(\d\d\d\d [A-Z]+ \d+\w+ \w+)|([A-Z] [A-Z]+ [A-Z]+ \d [A-Z]+ \/ [A-Z] [A-Z]+ [A-Z]+)|([A-Z]+ [A-Z]+ \/ \d+[A-Z]+ [A-Z]+ [A-Z]+)|(\d{4} \d{3}[A-Z]+ [A-Z]+ [A-Z]+)|(\d+ [A-Z] [A-Z]\d\d)
regex_location = r"( \d{1,5}. ([A-Z]|[1-9])+ [A-Z]+ (([A-Z]+)\b)?)|( [A-Z]+ [A-Z]+ [A-Z]+ \/ [A-Z]+ [A-Z]+ [A-Z]+)|( [A-Z]+ [A-Z]+ \/ [A-Z]+ [A-Z]+)|([A-Z] [A-Z]+ [A-Z]+ \/ \d+[A-Z]+ [A-Z]+ [A-Z]+)|([1-9]+[A-Z]+ [A-Z]+ [A-Z]+ \/ [A-Z]+ \d)|(\d\d\d\d [A-Z]+ \d+\w+ \w+)|([A-Z] [A-Z]+ [A-Z]+ \d [A-Z]+ \/ [A-Z] [A-Z]+ [A-Z]+)|([A-Z]+ [A-Z]+ \/ \d+[A-Z]+ [A-Z]+ [A-Z]+)|(\d{4} \d{3}[A-Z]+ [A-Z]+ [A-Z]+)|(\d+ [A-Z] [A-Z]\d\d)"
regex_nature = r"([A-Z][a-z]+\/[A-Z][a-z]+)|([A-Z][a-z]+ [A-Z][a-z]+\/[A-Z][a-z]+ [A-Z][a-z]+)|([A-Z][a-z]+ [A-Z][a-z]+ [A-Z][a-z]+ [A-Z][a-z]+)|([A-Z][a-z]+ [A-Z][a-z]+ [A-Z][a-z]+)|((MVA) [A-Z][a-z]+ [A-Z][a-z]+)|([A-Z][a-z]+ [A-Z][a-z]+)|([A-Z][a-z]+)"
regex_ori = r"(EMSSTAT)|([A-Z][A-Z]\d\d\d\d\d\d\d)|(14005)"

#(\w+\/\w+)|(\w+\/\w+ [A-Z]+)|([A-Z][a-z]+)


def store_data_from_pdf(pdf):
  count = 0
  s = 0
  f = 0
  for i in pdf:
    try:
      dt = re.search(regex_datetime, i)
      inc = re.search(regex_incident_num, i)
      loc = re.search(regex_location, i)
      nat = re.search(regex_nature, i)
      o = re.search(regex_ori, i)

      datetime = dt.group()
      incident_num = inc.group()
      location = loc.group()
      nature = nat.group()
      ori = o.group()

      add_to_database(datetime, incident_num, location, nature, ori)
      s = s + 1
    except:
      print("An exeption occured with: ", i)
      f = f + 1

    count = count + 1

  print(s, " Successes occurred")
  print(f, " Failures occurred")


store_data_from_pdf(p)


An exeption occured with:  Date / Time Incident Number Location Nature Incident ORI
An exeption occured with:  1/1/2023 0:20 2023-00000004TECHNOLOGY PL / E STATE HWY 9 
An exeption occured with:  HWYTraffic Stop OK0140200
An exeption occured with:  1/1/2023 0:32 2023-00000009 35.2057133333333;-97.447475 Contact a Subject OK0140200
An exeption occured with:  Daily Incident Summary (Public)
An exeption occured with:  1/1/2023 1:12 2023-00000020 FLOOD / MAIN Contact a Subject OK0140200
An exeption occured with:  1/1/2023 11:11 2023-00000021 179TH/DOUGLAS Fire Mutual Aid 14005
An exeption occured with:  1/1/2023 11:31 2023-00000088 1321 60TH AVE NE Animal Livestock OK0140200
An exeption occured with:  1/1/2023 13:53 2023-00000102 201-B W GRAY VPO Violation OK0140200
An exeption occured with:  1/1/2023 14:09 2023-00000107 35.2324983333333;-97.4057433333333 Traffic Stop OK0140200
An exeption occured with:  1/1/2023 14:56 2023-00000123STUBBEMAN AVE / W ROCK CREEK 
An exeption occured with:  R

In [82]:
def create_database():
  con = sqlite3.connect("normanpolice.db")
  cur = con.cursor()
  cur.execute("CREATE TABLE incidents(datetime, incident_num, location, nature, ori)")

In [83]:
def add_to_database(datetime, incident_num, location, nature, ori):
  con = sqlite3.connect("normanpolice.db")
  cur = con.cursor()
  cur.execute("INSERT INTO incidents (datetime, incident_num, location, nature, ori) values (?, ?, ?, ?, ?)", (datetime, incident_num, location, nature, ori))
  con.commit()



In [84]:
def print_database():
  con = sqlite3.connect("normanpolice.db")
  cur = con.cursor()
  res = cur.execute("SELECT * FROM incidents")
  res.fetchall()

In [87]:
print_database()

In [88]:
con = sqlite3.connect("normanpolice.db")
cur = con.cursor()
res = cur.execute("SELECT * FROM incidents")
res.fetchall()

[('5/5/5555 5:55',
  '5555555555',
  '555 north five',
  'avengers level threat',
  '5favie5555'),
 ('5/5/5555 5:55',
  '5555555555',
  '555 north five',
  'avengers level threat',
  '5favie5555'),
 ('5/5/5555 5:55',
  '5555555555',
  '555 north five',
  'avengers level threat',
  '5favie5555'),
 ('4/4/4444', '4432', 'south', 'its sooo bad', 'er443231'),
 ('1/1/2023 0:06',
  '2023-00000001',
  ' 2000 ANN BRANDEN BLVD',
  'Transfer',
  'EMSSTAT'),
 ('1/1/2023 0:15',
  '2023-00000001',
  ' 1432 24TH AVE SE',
  'Fireworks',
  'OK0140200'),
 ('1/1/2023 0:17',
  '2023-00000003',
  ' 3712 BURLINGTON DR ',
  'Open',
  'OK0140200'),
 ('1/1/2023 0:18',
  '2023-00000002',
  ' N FLOOD AVE / W ROCK CREEK',
  'Traffic',
  'OK0140200'),
 ('1/1/2023 0:18',
  '2023-00000011',
  ' 3802 BELLWOOD DR ',
  'Shots',
  'OK0140200'),
 ('1/1/2023 0:23',
  '2023-00000005',
  ' 1640 ALAMEDA ST ',
  'Traffic',
  'OK0140200'),
 ('1/1/2023 0:26', '2023-00000006', ' 508 ELM AVE ', 'Traffic', 'OK0140200'),
 ('1/1/202

In [ ]:
def Find(string):
 
    # findall() has been used
    # with valid conditions for urls in string
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex, string)
    return [x[0] for x in url]

In [ ]:
def incident_files(string):
  regex = r"(incident)"
  url = re.findall(regex, string)
  if not url:
    return
  else:
    return string